# imports

In [13]:
from io import BytesIO
from zipfile import ZipFile
import datetime
from datetime import date, timedelta
import os
import requests
import pandas as pd
import numpy as np
import time
import socket

import os
import requests
import pandas as pd
import numpy as np

from pystac_client import Client
from shapely.geometry import shape, Point

import geopandas as gpd
import matplotlib.pyplot as plt


from datetime import timedelta

from planetary_computer import sign
from stackstac import stack
import rasterio
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import datetime

import hvplot.pandas
import panel as pn
pn.extension('bokeh')

import random

from stackstac import stack
from planetary_computer import sign
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from odc.stac import load

import xarray as xr
import hvplot.xarray


/tmp/ipykernel_45000/542201738.py:36: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension('bokeh')


In [14]:
target_date = pd.to_datetime("2023-05-01").date()

# get ais data

In [15]:
fname = f"AIS_2023_{target_date.month:02d}_{target_date.day:02d}.zip"
url = f"https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2023/{fname}"

r = requests.get(url, timeout=60)
with ZipFile(BytesIO(r.content)) as z:
    csv_name = z.namelist()[0]
    with z.open(csv_name) as f:
        ais = pd.read_csv(f)
        success = True


# Clean
ais = ais[ais.TransceiverClass == 'A']                      #large cargo ships, not personal vessels
ais = ais[(ais.SOG > 1) & (ais.SOG < 80)]                   #drops ships in harbor, bad data
ais = ais[(ais.Length > 30) & (ais.Length < 400)]           #drops small ships, bad data
ais = ais.replace({'Heading': {511: np.nan}})               #heading 511 is nonrespondor, set to nan

# Sort
ais = ais.sort_values(by=['MMSI', 'BaseDateTime']).reset_index(drop=True)

# Drop weak tracks
mmsi_counts = ais.MMSI.value_counts()
active = mmsi_counts[mmsi_counts >= 5].index
ais = ais[ais["MMSI"].isin(active)].reset_index(drop=True)

#standardize date dtype
ais["BaseDateTime"] = pd.to_datetime(ais["BaseDateTime"], utc=True)

In [16]:
ais.sample()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass
322026,366808340,2023-05-01 14:18:33+00:00,37.77739,-122.59144,7.0,65.9,67.0,CALIFORNIA,IMO9245471,WDA4337,90.0,5.0,31.0,9.0,NaN,50.0,A


In [17]:
ais.Cargo.value_counts()

Cargo
70.0    250623
57.0    186650
80.0    141933
60.0    101986
31.0     60767
         ...  
6.0         83
59.0        44
94.0         9
46.0         9
93.0         5
Name: count, Length: 68, dtype: int64

In [18]:
ais.MMSI.value_counts().quantile(.1)

np.float64(21.0)

# cut down to intersections

In [19]:
api = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [20]:
results = api.search(
    collections=["sentinel-1-grd"],

    bbox=[-160, 4, -50, 50],
    datetime='2023-05-01',#str(target_date),
)

In [21]:
sentinel_passes = []
for item in results.get_all_items():
    geom = shape(item.geometry)
    dt = item.datetime
    sentinel_passes.append((geom, dt))
    
print(len(sentinel_passes))

/home/ec2-user/envs/ships-env/lib64/python3.9/site-packages/pystac_client/item_search.py:851: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


100


In [22]:
item

<Item id=S1A_IW_GRDH_1SDV_20230501T001325_20230501T001354_048333_05D01E>

In [23]:
geoms = [poly for poly, _ in sentinel_passes]
times = [dt for _, dt in sentinel_passes]

gdf = gpd.GeoDataFrame({'datetime': times}, geometry=geoms, crs='EPSG:4326')

In [24]:
gdf.hvplot(
    geo=True,
    tiles='CartoLight',
    line_color='blue',
    line_width=0.5,
    frame_width=900,
    frame_height=600
)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

In [25]:
# Prep sentinel GeoDataFrame
poly_list = []
time_list = []

filtered_passes = [
    (poly, dt) for poly, dt in sentinel_passes
    if dt.date() == target_date
]

poly_list = [poly for poly, _ in filtered_passes]
time_list = [dt for _, dt in filtered_passes]

sentinel_gdf = gpd.GeoDataFrame({'datetime': time_list}, geometry=poly_list, crs='EPSG:4326')

In [26]:
ais_gdf = gpd.GeoDataFrame(
    ais,
    geometry=gpd.points_from_xy(ais["LON"], ais["LAT"]),
    crs='EPSG:4326'
)

# We'll accumulate matching AIS rows into this list
matching_rows = []

# Define time window in seconds
TIME_WINDOW = timedelta(minutes=5)

# Iterate over sentinel polygons
for _, s2_row in sentinel_gdf.iterrows():
    poly = s2_row.geometry
    s2_time = s2_row.datetime

    # Filter AIS to time window first (fast)
    time_mask = (ais_gdf["BaseDateTime"] >= s2_time - TIME_WINDOW) & \
                (ais_gdf["BaseDateTime"] <= s2_time + TIME_WINDOW)
    candidate_ais = ais_gdf[time_mask]

    # Spatial filter: points inside this polygon
    inside_mask = candidate_ais.geometry.intersects(poly)
    intersecting = candidate_ais[inside_mask]

    if not intersecting.empty:
        matching_rows.append(intersecting)

# Concatenate results into one DataFrame
ais_intersecting = pd.concat(matching_rows, ignore_index=True)

In [27]:
len(ais_intersecting)

2602

In [28]:
len(ais_intersecting.MMSI.value_counts())

356

In [29]:
len(ais)

1121440

In [30]:
import geopandas as gpd
import pandas as pd
import hvplot.pandas  # Enables hvplot on pandas and geopandas objects

# Convert gulf_df into GeoDataFrame
ais_gdf = gpd.GeoDataFrame(
    ais_intersecting,
    geometry=gpd.points_from_xy(ais_intersecting["LON"], ais_intersecting["LAT"]),
    crs="EPSG:4326"
)

# Optional: filter AIS to a single day to avoid clutter
ais_gdf["BaseDateTime"] = pd.to_datetime(ais_gdf["BaseDateTime"])
ais_day = ais_gdf[ais_gdf["BaseDateTime"].dt.date == target_date]

# Interactive plot using hvplot
gdf_plot = gdf.hvplot(geo=True, line_color='blue', fill_alpha=0, line_width=0.5, title='Sentinel-2 Overpasses + AIS Ship Positions (2023-01-01)')
ais_plot = ais_day.hvplot(geo=True, color='red', size=2, alpha=0.5)

(gdf_plot * ais_plot).opts(
    width=800, height=500,
    xlabel='Longitude',
    ylabel='Latitude',
    tools=['hover'],
    show_grid=True
)


:Overlay
   .Polygons.I :Polygons   [Longitude,Latitude]
   .Points.I   :Points   [Longitude,Latitude]

# ship location extrapolation 

In [31]:
import numpy as np
from datetime import datetime

def project_ship_position(
    lat, lon, sog_knots, cog_deg, ais_time, sar_time
):
    """
    Projects the ship's position from AIS time to SAR image time using speed and heading.
    
    Args:
        lat (float): AIS latitude
        lon (float): AIS longitude
        sog_knots (float): Speed Over Ground (knots)
        cog_deg (float): Course Over Ground (degrees from North)
        ais_time (datetime): Timestamp of AIS ping (timezone-aware)
        sar_time (datetime): Timestamp of SAR image (timezone-aware)
    
    Returns:
        (adjusted_lat, adjusted_lon): Projected position at SAR image time
    """
    # Convert SOG to m/s (1 knot ≈ 0.51444 m/s)
    sog_mps = sog_knots * 0.51444

    # Time difference in seconds
    delta_t = (sar_time - ais_time).total_seconds()

    # If no movement or missing data
    if sog_knots == 0 or np.isnan(cog_deg):
        return lat, lon

    # Distance moved in meters
    distance_m = sog_mps * delta_t

    # Convert COG to radians
    cog_rad = np.deg2rad(cog_deg)

    # Displacement
    dx = distance_m * np.sin(cog_rad)  # East (lon)
    dy = distance_m * np.cos(cog_rad)  # North (lat)

    # Convert meters to degrees
    meters_per_deg_lat = 111320
    meters_per_deg_lon = 111320 * np.cos(np.deg2rad(lat))

    delta_lat = dy / meters_per_deg_lat
    delta_lon = dx / meters_per_deg_lon

    # Adjusted position
    adjusted_lat = lat + delta_lat
    adjusted_lon = lon + delta_lon

    return adjusted_lat, adjusted_lon


# get sat image

In [32]:
api = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [33]:
row = ais_intersecting.iloc[random.randint(0,len(ais_intersecting))]
print(f"Selected MMSI: {row['MMSI']} @ {row['BaseDateTime']}")

ais_point = Point(row["LON"], row["LAT"])
time_buffer = timedelta(minutes=15)

Selected MMSI: 477133800 @ 2023-05-01 02:04:29+00:00


In [34]:
crop_buffer = .02 # .01 = ~1 km

In [35]:
search = api.search(
    collections=["sentinel-1-grd"],
    bbox=[row["LON"] - crop_buffer, row["LAT"] - crop_buffer, row["LON"] + crop_buffer, row["LAT"] + crop_buffer],
    datetime=f"{(row['BaseDateTime'] - time_buffer).isoformat()}/{(row['BaseDateTime'] + time_buffer).isoformat()}",
    limit=5,
    query={
        "sar:instrument_mode": {"eq": "IW"},
    }
)

items = list(search.get_items())

tile_geom = shape(items[0].geometry)

print("Tile bounds:", tile_geom.bounds)
print("AIS point inside tile?", tile_geom.contains(ais_point))

/home/ec2-user/envs/ships-env/lib64/python3.9/site-packages/pystac_client/item_search.py:836: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


Tile bounds: (-124.6444356, 47.0019169, -120.9288102, 48.8877295)
AIS point inside tile? True


In [36]:
item = sign(items[0])

arr = load(
    [item],
    bands=['vv', 'vh'],
    crs="EPSG:4326",  # Important: forces lat/lon coordinates
    resolution=0.0001,  # About 10m at equator
    bbox=(row["LON"] - crop_buffer, row["LAT"] - crop_buffer, row["LON"] + crop_buffer, row["LAT"] + crop_buffer),
    groupby="solar_day"
)

In [37]:
print(arr)

<xarray.Dataset> Size: 1MB
Dimensions:      (latitude: 401, longitude: 401, time: 1)
Coordinates:
  * latitude     (latitude) float64 3kB 48.3 48.3 48.3 ... 48.26 48.26 48.26
  * longitude    (longitude) float64 3kB -123.9 -123.9 -123.9 ... -123.8 -123.8
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 8B 2023-05-01T02:02:48.764228
Data variables:
    vv           (time, latitude, longitude) float32 643kB 85.0 87.0 ... 95.0
    vh           (time, latitude, longitude) float32 643kB 40.0 40.0 ... 33.0


In [57]:
mode = 'vh'

In [58]:
from datetime import datetime
import numpy as np

# Step 1: Base lat/lon from AIS row
lat = row["LAT"]
lon = row["LON"]
sog = row["SOG"]
cog = row["COG"]
ais_time = row["BaseDateTime"]

# Step 2: Get SAR acquisition time from the array
sar_time = pd.to_datetime(arr[mode].time.values[0]).tz_localize("UTC")

# Step 3: Project position using the function
adj_lat, adj_lon = project_ship_position(lat, lon, sog, cog, ais_time, sar_time)

# Step 4: Create slices around projected position
lat_slice = slice(adj_lat + crop_buffer, adj_lat - crop_buffer)
lon_slice = slice(adj_lon - crop_buffer, adj_lon + crop_buffer)

# Step 5: Crop the SAR array to center on projected ship position
crop = arr[mode].sel(
    latitude=lat_slice,
    longitude=lon_slice
).isel(time=0)



In [59]:
arr[mode].shape

(1, 401, 401)

In [60]:
crop.shape

(392, 304)

In [61]:


img = crop


vmin, vmax = np.percentile(img.values[~np.isnan(img.values)], (2, 98))
img_clipped = img.clip(min=vmin, max=vmax)

plot = img_clipped.hvplot.image(
    x='longitude',
    y='latitude',
    cmap='gray',
    frame_width=600,
    frame_height=600,
    title=f"Sentinel-1 {mode} @ {lat:.2f}, {lon:.2f}",
    invert=True
)

plot 

:Image   [longitude,latitude]   (vh)

In [62]:
row[['MMSI', 'VesselName', 'SOG', 'Heading', 'Length']]

MMSI              477133800
VesselName    HAINAN ISLAND
SOG                    14.0
Heading               279.0
Length                177.0
Name: 1399, dtype: object